<a href="https://colab.research.google.com/github/Karaneeshwar/Basic-Recommendation-system/blob/main/2022506020_Jai_Karaneesh_Sundar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 1.6 MB/s eta 0:00:00


In [2]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [3]:
# Load ratings and movies data.
ratings = tfds.load('movielens/100k-ratings', split="train")
movies = tfds.load('movielens/100k-movies', split="train")

# Select basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "rating": x["user_rating"]
})
movies = movies.map(lambda x: x["movie_title"])

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/100000 [00:00<?, ? examples/s]

...

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1682 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/100k-movies/incomplete.E3IBS1_0.1.1/movielens-train.tfrecord*...…

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-movies/0.1.1. Subsequent calls will reuse this data.


In [4]:
# Setup StringLookup layers.
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

In [5]:
# Define the RNN-based collaborative filtering model.
class RNNCollaborativeFilteringModel(tfrs.Model):

    def __init__(self, user_model: tf.keras.layers.Layer, movie_model: tf.keras.layers.Layer, task: tfrs.tasks.Retrieval):
        super().__init__()

        # Set up user and movie models.
        self.user_model = user_model
        self.movie_model = movie_model

        # Define the RNN layers.
        self.user_rnn = tf.keras.layers.SimpleRNN(64)
        self.movie_rnn = tf.keras.layers.SimpleRNN(64)

        # Set up a retrieval task.
        self.task = task

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        # Compute user and movie embeddings using RNN.
        user_embeddings = self.user_model(features["user_id"])
        movie_embeddings = self.movie_model(features["movie_title"])

        # Expand dimensions to fit RNN input requirements.
        user_embeddings = tf.expand_dims(user_embeddings, axis=1)
        movie_embeddings = tf.expand_dims(movie_embeddings, axis=1)

        # Process embeddings through RNN layers.
        user_rnn_output = self.user_rnn(user_embeddings)
        movie_rnn_output = self.movie_rnn(movie_embeddings)

        # Compute retrieval task loss.
        return self.task(user_rnn_output, movie_rnn_output, compute_metrics=False)

In [6]:
# Define user and movie models with StringLookup and Embedding layers.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 64)
])

# Define retrieval task.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
))

In [7]:
# Create RNN-based collaborative filtering model.
model = RNNCollaborativeFilteringModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train the model for 3 epochs.
model.fit(ratings.batch(4096), epochs=3)

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title))))

# Get recommendations for user "42".
_, titles = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {titles[0, :3]}")


Epoch 1/3
25/25 [==============================] - 15s 467ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 62378.0323 - regularization_loss: 0.0000e+00 - total_loss: 62378.0323
Epoch 2/3
25/25 [==============================] - 14s 566ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 43242.3328 - regularization_loss: 0.0000e+00 - total_loss: 43242.3328
Epoch 3/3
25/25 [==============================] - 13s 507ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 

In [8]:
class NeuralCollaborativeFilteringModel(tfrs.Model):

    def __init__(self, user_model: tf.keras.Model, movie_model: tf.keras.Model, task: tfrs.tasks.Retrieval):
        super().__init__()

        # Set up user and movie representations.
        self.user_model = user_model
        self.movie_model = movie_model

        # Define the interaction model.
        self.interaction_model = tf.keras.Sequential([
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(1)  # Output a single score
        ])

        # Set up a retrieval task.
        self.task = task

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        # Compute embeddings.
        user_embeddings = self.user_model(features["user_id"])
        movie_embeddings = self.movie_model(features["movie_title"])

        # Concatenate embeddings.
        concatenated_embeddings = tf.concat([user_embeddings, movie_embeddings], axis=1)

        # Compute interaction score.
        interaction_scores = self.interaction_model(concatenated_embeddings)

        # Compute loss.
        return self.task(user_embeddings, movie_embeddings, compute_metrics=False) + tf.reduce_mean(interaction_scores)

In [9]:
# Create a retrieval model.
model = NeuralCollaborativeFilteringModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=3)

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title))))

# Get some recommendations.
_, titles = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {titles[0, :3]}")

Epoch 1/3
25/25 [==============================] - 18s 705ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: -553783157.5183 - regularization_loss: 0.0000e+00 - total_loss: -553783157.5183
Epoch 2/3
25/25 [==============================] - 13s 495ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: -7372124238.7692 - regularization_loss: 0.0000e+00 - total_loss: -7372124238.7692
Epoch 3/3
25/25 [==============================] - 13s 498ms/step - factorized_top_k/top_1_categorical

In [10]:
class ContentBasedFilteringModel(tfrs.Model):
    def __init__(self, movie_model: tf.keras.Model, task: tfrs.tasks.Retrieval):
        super().__init__()

        # Set up movie representations.
        self.movie_model = movie_model

        # Set up a retrieval task.
        self.task = task

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        # Compute movie embeddings.
        movie_embeddings = self.movie_model(features["movie_title"])
        highest_rated_embeddings = self.movie_model(features["highest_rated_movie"])

        # Compute loss.
        return self.task(highest_rated_embeddings, movie_embeddings)

In [11]:
# Function to get the highest rated movie for a specific user.
def get_highest_rated_movie(user_id: str):
    user_ratings = ratings.filter(lambda x: x["user_id"] == user_id)
    highest_rated_movie = None
    highest_rating = -1
    for rating in user_ratings:
        movie_title = rating['movie_title'].numpy().decode('utf-8')
        user_rating = rating['rating'].numpy()
        if user_rating > highest_rating:
            highest_rated_movie = movie_title
            highest_rating = user_rating
    return highest_rated_movie

# Example user ID.
user_id = "42"

# Get the highest rated movie for the user.
highest_rated_movie = get_highest_rated_movie(user_id)

if highest_rated_movie is None:
    raise ValueError(f"No ratings found for user {user_id}")

In [12]:
# Create a content-based filtering model.
model = ContentBasedFilteringModel(movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Prepare the training dataset.
training = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "highest_rated_movie": tf.constant(highest_rated_movie, dtype=tf.string)
}).batch(4096)

# Train for 3 epochs.
model.fit(training, epochs=3)

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.movie_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title))))

# Assuming movie_model expects a batched input of movie titles
highest_rated_movie_batched = tf.constant([highest_rated_movie], dtype=tf.string)

# Get recommendations based on the user's highest rated movie.
_, titles = index(highest_rated_movie_batched)
print(f"Top 3 recommendations based on '{highest_rated_movie}': {titles[0, :3]}")


Epoch 1/3
25/25 [==============================] - 30s 1s/step - factorized_top_k/top_1_categorical_accuracy: 8.0000e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0025 - factorized_top_k/top_10_categorical_accuracy: 0.0115 - factorized_top_k/top_50_categorical_accuracy: 0.1154 - factorized_top_k/top_100_categorical_accuracy: 0.2109 - loss: 5794648.2500 - regularization_loss: 0.0000e+00 - total_loss: 5794648.2500
Epoch 2/3
25/25 [==============================] - 28s 1s/step - factorized_top_k/top_1_categorical_accuracy: 2.9000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0062 - factorized_top_k/top_10_categorical_accuracy: 0.0177 - factorized_top_k/top_50_categorical_accuracy: 0.0950 - factorized_top_k/top_100_categorical_accuracy: 0.1882 - loss: 4683461.9423 - regularization_loss: 0.0000e+00 - total_loss: 4683461.9423
Epoch 3/3
25/25 [==============================] - 28s 1s/step - factorized_top_k/top_1_categorical_accuracy: 1.1000e-04 - factorized_top_k/top_5_categori